# Intro/Imports
This is going to be a more granular, customizable neural net.

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import joblib
import pathlib as path
import copy

# Data Read-in
unchanged from neuralnet.ipynb; this is non-negotiable. 
For any of the user changes to work, I need to know the shape of the data; so this split will be universal. 

In [11]:
rawData = pd.read_csv('STEMVisualsSynthData.csv', header=0)
#remove unneeded column
rawData.drop('Index_within_Experiment', axis = 1, inplace = True)
#X is inputs--the three Concentrations, F_in, I0 (light intensity), and c_N_in (6)
X = rawData[['Time', 'C_X', 'C_N', 'C_L', 'F_in', 'C_N_in', 'I0']]
Y = X.copy(deep=True)
#drop unnecessary rows in Y
Y.drop('F_in', axis = 1, inplace = True)
Y.drop('C_N_in', axis = 1, inplace = True)
Y.drop('I0', axis = 1, inplace = True)
Y.drop('Time', axis = 1, inplace = True)
#Y vals should be X concentrations one timestep ahead, so remove the first index
Y.drop(index=0, inplace=True)
#To keep the two consistent, remove the last index of X
X.drop(index=19999, inplace=True)
#separate the times out into their own little thing for later use

C:\Users\kenda\AppData\Local\Temp\ipykernel_27940\3039068683.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(index=19999, inplace=True)


# Train/Test Split
Keeping it simple stupid, there's only 1 split happening here. User chooses the percentage of data to hold out for testing.

In [12]:
#defaults
train_ratio = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1 - train_ratio)

#function to call whenever wanting to change the train/test split
def trainSplit():
    train_ratio = float(input("What percentage of data to holdout for testing? (default 0.2) "))
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1 - train_ratio)

#Separate Time out--we don't want this as a feature the model learns (since it's already incorporated in by how X and Y are structured!)
#however, having the time values will be useful for plotting later
XTrainTime = X_train.pop('Time')
XTestTime = X_test.pop('Time')

# Preprocessing
Another thing I'm going to just have to have happen backend without the user's input.

In [13]:
#for both x and y, create a min max scaler
mmscalerX = preprocessing.MinMaxScaler()
#we'll fit and transform based on training data
X_train_minmax = mmscalerX.fit_transform(X_train)
#transform testing data based on training data
X_test_minmax = mmscalerX.transform(X_test)
#same process as above for Y
mmscalerY = preprocessing.MinMaxScaler()
Y_train_minmax = mmscalerY.fit_transform(Y_train)
Y_test_minmax = mmscalerY.transform(Y_test)

# Tensor Creation

In [14]:
#put our data into tensors
X_train = torch.tensor(X_train_minmax, dtype=torch.float32)
y_train = torch.tensor(Y_train_minmax, dtype=torch.float32)
X_test = torch.tensor(X_test_minmax, dtype=torch.float32)
y_test = torch.tensor(Y_test_minmax, dtype=torch.float32)

# Model Creation
Here's where the majority of new stuff comes in--we need to let the user have control over a lot of things.
- Num Layers: How many patterns of Linear/activation to use? range: 1-5
- Num Nodes: How many hidden nodes per layer? range: 5-20
- Optimizer: How does the model update its weights? choices: ADAM, SGD
- Loss: How does the model measure accuracy? choices: L1, MSE, LKDivLoss
- Learning Rate: How greatly can the optimizer change weights? range: 0.00001-0.1
- Epochs: How many times to run the training data through the model? range: 10-1000

In [18]:
numLayers = int(input("How many layers? (default 2, range 1-5) "))
numNodes = int(input("How many nodes per layer? (default 10, range 5-20) "))
#set up the model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], numNodes),
    nn.ReLU(),
    nn.Linear(numNodes, numNodes),
    nn.ReLU(),
    nn.Linear(numNodes, numNodes),
    nn.ReLU(),
    nn.Linear(numNodes, y_train.shape[1]),
)
lossChoice = int(input("Which loss function? (1: L1, 2: MSE, 3: KLDiv) "))
#set up the loss function
match lossChoice:
    case 1:
        loss_fn = nn.L1Loss()
    case 2:
        loss_fn = nn.MSELoss()
    case 3:
        loss_fn = nn.KLDivLoss()
    case _:
        print("Invalid loss function choice. Defaulting to L1.")
        loss_fn = nn.L1Loss()

#set up the optimizer
optimChoice = int(input("Which optimizer? (1: SGD, 2: Adam) "))
lr = float(input("Learning rate? (default 0.01) "))
match optimChoice:
    case 1:
        optimizer = optim.SGD(model.parameters(), lr=lr)
    case 2:
        optimizer = optim.Adam(model.parameters(), lr=lr)
    case _:
        print("Invalid optimizer choice. Defaulting to SGD.")
        optimizer = optim.SGD(model.parameters(), lr=lr)



# Training
Mostly unchanged. User can change:
- num of Epochs, range: 10-1000
- Batch size, range: 100-1000

In [19]:
# training parameters
n_epochs = int(input("How many epochs? (default 100) "))
batch_size = int(input("Batch size? (default 100) "))
batch_start = torch.arange(0, len(X_train), batch_size)
 
# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []
 
# training loop
for epoch in range(n_epochs):
    model.train()
    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]
            # forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(mse=float(loss))
    # evaluate accuracy at end of each epoch
    model.eval()
    y_pred = model(X_test)
    mse = loss_fn(y_pred, y_test)
    mse = float(mse)
    history.append(mse)
    if mse < best_mse:
        best_mse = mse
        best_weights = copy.deepcopy(model.state_dict())
 
# restore model and return best accuracy
model.load_state_dict(best_weights)

<All keys matched successfully>

# Loss Graph
(unchanged)

In [ ]:
# restore model and return best accuracy
model.load_state_dict(best_weights)
print("MSE: %.2f" % best_mse)
print("RMSE: %.2f" % np.sqrt(best_mse))
plt.plot(history)
plt.xlabel("Epochs")
plt.ylabel("MSE")
plt.show()